In [ ]:
import matplotlib.pyplot as plt 
import matplotlib

import sys
print(sys.version)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
import numpy as np

from palettable.tableau import Tableau_10
import umap
import numpy as np
import statsmodels.api as sm

pd.set_option("display.max_rows", 400)
import pickle as pkl
import pyodbc

%matplotlib inline

connection_string = ("DRIVER=/home/bkb12/Staging/lib/libtdsodbc.so; " + 
                     "SERVER=10.117.35.36; " +
                     "PORT=1401; " +
                     "UID=bkb12; " +
                     "PWD=21Prince; " +
                     "UseNTLMv2=yes;TDS_Version=8.0;")
                        
cn = pyodbc.connect(connection_string, autocommit=True)

In [ ]:
query = """
SELECT ct.EMPI, ct.CoverageStart, ct.CoverageEnd, d.FirstDiag, DATEDIFF(day, ct.CoverageStart, d.FirstDiag) DaysBeforeDiag,
       DATEDIFF(day, d.FirstDiag, ct.CoverageEnd) DaysAfterDiag
FROM
(SELECT EMPI, CAST(FirstEnc as date) CoverageStart, CAST(LastEnc as date) CoverageEnd FROM pd.dbo.CoverageTime) ct
JOIN (SELECT EMPI, MIN(CAST(Date as date)) FirstDiag FROM pd.dbo.Dia WHERE Code IN ('332.0', 'G20') GROUP BY EMPI) d
ON ct.EMPI = d.EMPI
WHERE DATEDIFF(day, ct.CoverageStart, d.FirstDiag) > 365
"""

df = pd.read_sql(query, cn)


In [ ]:
import seaborn as sns
from lifelines.utils import median_survival_times

sns.set_context("poster")

from lifelines import KaplanMeierFitter
plt.figure(figsize=(12, 8))
kmf = KaplanMeierFitter(label='Time from Diagnosis Until Last Encounter')
kmf.fit(df['DaysAfterDiag']) 
kmf.survival_function_
kmf.cumulative_density_

median_ = kmf.median_survival_time_
median_confidence_interval_ = median_survival_times(kmf.confidence_interval_)

ax = kmf.plot_survival_function()
ax.set_xlabel("Days from Initial PD Diagnosis")
plt.tight_layout()
